In [1]:
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings('ignore')

import requests
import pandas as pd
import numpy as np
import datetime as dt
import os
from scipy.stats import norm as norm

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')
from yahoo_query import *
pd.options.display.float_format = '{:,.6f}'.format
%matplotlib inline
os.chdir('D:\Options Data\CBOE Manual Downloads')

In [218]:
filename = 'spx_2019-02-08-12-49.dat'

vix = yahoo_query('^VIX',dt.datetime.today())
vix.minute_query()
vix = vix.minute_prices.dropna()

options_chain = pd.read_csv(filename, sep=',', header=0, skiprows=2)

i = 0
for line in open(filename,'r'):
    curr_line = [x.strip() for x in line.split(',')]
    if i == 0:
        curr_spx = float(curr_line[1])
    
    if i == 1:
        curr_time = curr_line[0].replace('@','').replace('  ',' ').replace(' ET','')
        curr_time = dt.datetime.strptime(curr_time, '%b %d %Y %H:%M')
    if i == 1:
        break
    i += 1
    
options_chain['Expiration Date'] = pd.to_datetime(options_chain['Expiration Date'])

universal_columns = ['Expiration Date','Strike']
call_columns = list(filter(lambda x: x in universal_columns or '1' not in x, options_chain.columns.tolist()))
put_columns = list(filter(lambda x: x in universal_columns or '1' in x, options_chain.columns.tolist()))

calls = options_chain[call_columns]
del calls['Calls'], calls['Puts']

a = (calls['IV']**2)/2
b = -calls['IV']*norm.ppf(calls['Delta'])
c = np.log(curr_spx/calls['Strike'])

calls['time_remaining'] = ((-b + np.sqrt(b**2 - 4*a*c))/(2*a))**2

puts = options_chain[put_columns]
puts.columns = [x.replace('.1','') for x in puts.columns.tolist()]

commissions = 0.0266

def create_spreads(spreads, contract_type = 'puts', net_delta_threshold = 0.275):

    spreads['Net Delta'] = abs(spreads['Short Delta'] - spreads['Long Delta'])
    spreads = spreads[spreads['Net Delta'] <= net_delta_threshold]
    spreads['Credit'] = spreads['Short Bid'] - spreads['Long Ask'] - commissions
    spreads['Required_Contracts'] = round(0.5/spreads['Net Delta'])
    spreads['Total Premo'] = spreads.Credit*100*spreads.Required_Contracts
    
    if contract_type == 'puts':
        spreads['Max Loss'] = (spreads['Long Strike'] - spreads['Short Strike'])*100*spreads.Required_Contracts + spreads['Total Premo']
        spreads['Break Even'] = spreads['Short Strike'] - spreads.Credit

        spreads['SPX Equiv Lower Loss'] = (spreads['Long Strike'] - curr_spx)*100
        spreads['SPX Equiv Break Even Loss'] = (spreads['Break Even'] - curr_spx)*100
        spreads['SPX Equiv Gain Level'] = curr_spx + spreads['Total Premo']/100

        spreads['Prob of Better Than ES'] = norm.cdf(spreads['SPX Equiv Gain Level']/curr_spx - 1,0,spreads['Weekly IV']) - norm.cdf(spreads['Break Even']/curr_spx - 1,0,spreads['Weekly IV']) + norm.cdf(spreads['Long Strike']/curr_spx - 1,0,spreads['Weekly IV'])

    if contract_type == 'calls':
        spreads['Max Loss'] = (spreads['Short Strike'] - spreads['Long Strike'])*100*spreads.Required_Contracts + spreads['Total Premo']
        spreads['Break Even'] = spreads['Short Strike'] + spreads.Credit

        spreads['SPX Equiv Lower Loss'] = (curr_spx - spreads['Long Strike'])*100
        spreads['SPX Equiv Break Even Loss'] = -(spreads['Break Even'] - curr_spx)*100
        spreads['SPX Equiv Gain Level'] = curr_spx - spreads['Total Premo']/100

        spreads['Prob of Better Than ES'] = norm.cdf(spreads['Break Even']/curr_spx - 1,0,spreads['Weekly IV']) - norm.cdf(spreads['SPX Equiv Gain Level']/curr_spx - 1,0,spreads['Weekly IV']) + 1 - norm.cdf(spreads['Long Strike']/curr_spx - 1,0,spreads['Weekly IV'])
    
    spreads['EV'] = np.nan
    spreads['Win Prob'] = np.nan
    stepsize = 0.05
    
    for idx, row in spreads.iterrows():
        
        if contract_type == 'puts':
            ev_df = pd.DataFrame({'SPX': np.arange(row['Long Strike'], row['Short Strike'] + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row['Short Strike'] + stepsize)]
            ev_df['PnL'] = ev_df['SPX'] - row['Short Strike'] + row.Credit
            
        if contract_type == 'calls':
            ev_df = pd.DataFrame({'SPX': np.arange(row['Short Strike'], row['Long Strike'] + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row['Long Strike'] + stepsize)]
            ev_df['PnL'] = row['Short Strike'] - ev_df['SPX'] + row.Credit
    
        ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,row['Weekly IV'])
        lb = ev_df.loc[0,'Prob']
        ub = ev_df.loc[len(ev_df) - 1,'Prob']

        ev_df['Prob'] = ev_df['Prob'].diff()
        ev_df.loc[0,'Prob'] = lb
        ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

        ev_df['EV'] = ev_df.Prob*ev_df.PnL

        total_ev = sum(ev_df['EV'])
        win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

        spreads.loc[idx, 'EV'] = total_ev
        spreads.loc[idx, 'Win Prob'] = win_prob

    
    return spreads

In [219]:
day_diff = 2

bid_threshold = 0.5

net_delta_threshold = 0.275

weekly_puts = puts[(puts['Expiration Date'] - dt.datetime.today()).dt.days == day_diff]
weekly_puts = weekly_puts[(weekly_puts.Strike <= curr_spx) & 
                          (weekly_puts['Bid'] >= bid_threshold)].sort_values('Strike', ascending = False)
weekly_puts = weekly_puts.drop_duplicates(subset = ['Strike'], keep = 'last').reset_index(drop = True)

weekly_calls = calls[(calls['Expiration Date'] - dt.datetime.today()).dt.days == day_diff]
weekly_calls = weekly_calls[(weekly_calls.Strike >= curr_spx) & 
                            (weekly_calls['Bid'] >= bid_threshold)].sort_values('Strike', ascending = True)
weekly_calls = weekly_calls.drop_duplicates(subset = ['Strike'], keep = 'last').reset_index(drop = True)

time_remaining = weekly_calls.time_remaining.mean()

put_spreads_list = []
for idx, row in weekly_puts.iterrows():
    put_spreads = weekly_puts[weekly_puts.Strike < row.Strike][['Expiration Date','Strike','Ask','Delta']]
    if len(put_spreads) < 3:
        continue
    else:
        put_spreads.columns = ['Expiration Date','Long Strike','Long Ask', 'Long Delta']
        put_spreads['Short Strike'] = row.Strike
        put_spreads['Short Bid'] = row.Bid
        put_spreads['Short Delta'] = row.Delta
        put_spreads['Weekly IV'] = (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining)
    put_spreads_list.append(create_spreads(put_spreads, 'puts', net_delta_threshold))
    
put_spreads = pd.concat(put_spreads_list, axis = 0).dropna().reset_index(drop = True)
put_spreads = put_spreads[(put_spreads['Total Premo'] > weekly_puts.loc[0,'Bid']*100) & 
                          (put_spreads['Prob of Better Than ES'] > 0.5) & 
                          (put_spreads['Max Loss'] > -5000)].reset_index(drop = True)
put_spreads['Max Loss Differences'] = put_spreads['Max Loss'] - put_spreads['SPX Equiv Lower Loss']

put_spreads = put_spreads.sort_values('EV', ascending = False).reset_index(drop = True)
put_spreads[['Expiration Date','Long Strike','Short Strike','Required_Contracts','Credit','Total Premo','Max Loss','EV','Win Prob']]

,Expiration Date,Long Strike,Short Strike,Required_Contracts,Credit,Total Premo,Max Loss,EV,Win Prob
0,2019-02-11,"2,685.000000","2,690.000000",7.000000,1.473400,"1,031.380000","-2,468.620000",-0.905822,0.514205
1,2019-02-11,"2,670.000000","2,680.000000",5.000000,2.073400,"1,036.700000","-3,963.300000",-1.540602,0.612412


In [220]:
call_spreads_list = []
for idx, row in weekly_calls.iterrows():
    call_spreads = weekly_calls[weekly_calls.Strike > row.Strike][['Expiration Date','Strike','Ask','Delta']]
    if len(call_spreads) < 3:
        continue
    else:
        call_spreads.columns = ['Expiration Date','Long Strike','Long Ask', 'Long Delta']
        call_spreads['Short Strike'] = row.Strike
        call_spreads['Short Bid'] = row.Bid
        call_spreads['Short Delta'] = row.Delta
        call_spreads['Weekly IV'] = (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining)
    call_spreads_list.append(create_spreads(call_spreads, 'calls', net_delta_threshold))

call_spreads = pd.concat(call_spreads_list, axis = 0).dropna().reset_index(drop = True)
call_spreads = call_spreads[(call_spreads['Total Premo'] > weekly_calls.loc[0,'Bid']*100) & 
                            (call_spreads['Prob of Better Than ES'] > 0.5) &
                            (call_spreads['Max Loss'] > -5000)].reset_index(drop = True)
call_spreads['Max Loss Differences'] = call_spreads['Max Loss'] - call_spreads['SPX Equiv Lower Loss']

call_spreads = call_spreads.sort_values('EV', ascending = False).reset_index(drop = True)
call_spreads[['Expiration Date','Long Strike','Short Strike','Required_Contracts','Credit','Total Premo','Max Loss','EV','Win Prob']]

,Expiration Date,Long Strike,Short Strike,Required_Contracts,Credit,Total Premo,Max Loss,EV,Win Prob
0,2019-02-11,"2,700.000000","2,695.000000",6.000000,1.773400,"1,064.040000","-1,935.960000",-0.378237,0.562689
1,2019-02-11,"2,710.000000","2,705.000000",7.000000,1.073400,751.380000,"-2,748.620000",-0.628521,0.646965
2,2019-02-11,"2,705.000000","2,700.000000",6.000000,1.273400,764.040000,"-2,235.960000",-0.649425,0.604082
3,2019-02-11,"2,705.000000","2,695.000000",3.000000,3.373400,"1,012.020000","-1,987.980000",-0.701257,0.577510
4,2019-02-11,"2,710.000000","2,700.000000",3.000000,2.673400,802.020000,"-2,197.980000",-0.951562,0.616747
5,2019-02-11,"2,715.000000","2,705.000000",4.000000,1.923400,769.360000,"-3,230.640000",-1.270078,0.654386


In [221]:
stepsize = 0.05

weekly_puts['EV'] = np.nan
weekly_puts['Win Prob'] = np.nan

for idx, row in weekly_puts.iterrows():
    ev_df = pd.DataFrame({'SPX': np.arange(0, row['Strike'] + stepsize, stepsize)})
    ev_df = ev_df[(ev_df['SPX'] < row['Strike'] + stepsize)]
    ev_df['PnL'] = (row['Strike'] - ev_df['SPX'] - row.Ask)*100

    ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,
                             (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining))
    lb = ev_df.loc[0,'Prob']
    ub = ev_df.loc[len(ev_df) - 1,'Prob']

    ev_df['Prob'] = ev_df['Prob'].diff()
    ev_df.loc[0,'Prob'] = lb
    ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

    ev_df['EV'] = ev_df.Prob*ev_df.PnL

    total_ev = sum(ev_df['EV'])
    win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])
    
    weekly_puts.loc[idx, 'EV'] = total_ev
    weekly_puts.loc[idx, 'Win Prob'] = win_prob
    
weekly_puts

,Expiration Date,Strike,Last Sale,Net,Bid,Ask,Vol,IV,Delta,Gamma,Open Int,EV,Win Prob
0,2019-02-11,"2,690.000000",10.000000,2.050000,9.900000,10.400000,1039,0.100900,-0.500400,0.015800,1246,647.680190,0.402110
1,2019-02-11,"2,685.000000",8.000000,1.400000,8.000000,8.400000,356,0.104700,-0.424500,0.014900,181,609.730415,0.374997
2,2019-02-11,"2,680.000000",6.850000,1.400000,6.500000,6.800000,311,0.109000,-0.356900,0.013700,959,555.124600,0.345006
3,2019-02-11,"2,675.000000",5.450000,0.900000,5.200000,5.500000,367,0.112900,-0.297400,0.012200,254,493.374542,0.313864
4,2019-02-11,"2,670.000000",4.210000,0.460000,4.200000,4.400000,1734,0.116800,-0.246200,0.010800,2594,433.693252,0.281601
5,2019-02-11,"2,665.000000",3.150000,0.050000,3.300000,3.600000,3513,0.120700,-0.202700,0.009300,2077,365.034868,0.249154
6,2019-02-11,"2,660.000000",2.550000,-0.050000,2.600000,2.800000,168,0.123400,-0.164100,0.008000,597,316.111246,0.218811
7,2019-02-11,"2,655.000000",1.950000,-0.225000,2.050000,2.200000,508,0.126400,-0.132700,0.006800,1277,265.476035,0.189431
8,2019-02-11,"2,650.000000",1.650000,-0.175000,1.650000,1.750000,920,0.130200,-0.107900,0.005700,1368,216.546244,0.162035
9,2019-02-11,"2,645.000000",1.300000,-0.225000,1.300000,1.350000,554,0.132900,-0.086100,0.004700,1030,177.664255,0.136582


In [217]:
weekly_calls['EV'] = np.nan
weekly_calls['Win Prob'] = np.nan

for idx, row in weekly_calls.iterrows():
    ev_df = pd.DataFrame({'SPX': np.arange(row['Strike'], curr_spx*1.20, stepsize)})
    ev_df['PnL'] = (ev_df['SPX'] - row['Strike'] - row.Ask)*100

    ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0,
                             (vix.reset_index().loc[len(vix) - 1, '^VIX_close']/100)*np.sqrt(time_remaining))
    lb = ev_df.loc[0,'Prob']
    ub = ev_df.loc[len(ev_df) - 1,'Prob']

    ev_df['Prob'] = ev_df['Prob'].diff()
    ev_df.loc[0,'Prob'] = lb
    ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub

    ev_df['EV'] = ev_df.Prob*ev_df.PnL

    total_ev = sum(ev_df['EV'])
    win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])
    
    weekly_calls.loc[idx, 'EV'] = total_ev
    weekly_calls.loc[idx, 'Win Prob'] = win_prob
    
weekly_calls

,Expiration Date,Last Sale,Net,Bid,Ask,Vol,IV,Delta,Gamma,Open Int,Strike,time_remaining,EV,Win Prob
0,2019-02-08,5.000000,-19.650000,5.200000,5.600000,6950,0.131500,0.404400,0.020700,2946,"2,710.000000",0.002051,150.536471,0.335651
1,2019-02-08,3.250000,-17.200000,3.300000,3.800000,6515,0.129700,0.303000,0.018900,2486,"2,715.000000",0.002406,136.894890,0.285254
2,2019-02-08,2.200000,-14.600000,2.100000,2.400000,8274,0.129200,0.215300,0.015900,2683,"2,720.000000",0.002529,124.258018,0.233272
3,2019-02-08,1.400000,-12.150000,1.250000,1.500000,8126,0.129500,0.145500,0.012400,3790,"2,725.000000",0.002588,98.242919,0.181079
4,2019-02-08,0.840000,-9.710000,0.750000,0.950000,7887,0.131900,0.096600,0.009100,3168,"2,730.000000",0.002622,68.356402,0.133491
5,2019-02-08,0.550000,-7.550000,0.450000,0.550000,2893,0.133400,0.061000,0.006400,989,"2,735.000000",0.002649,48.652079,0.094167
6,2019-02-08,0.250000,-5.750000,0.250000,0.400000,6767,0.138400,0.040800,0.004400,3831,"2,740.000000",0.002665,23.336146,0.062740
7,2019-02-08,0.220000,-4.230000,0.150000,0.300000,2640,0.144700,0.028400,0.003200,2990,"2,745.000000",0.002678,7.227261,0.039969
8,2019-02-08,0.150000,-3.000000,0.100000,0.150000,5601,0.145900,0.016700,0.002000,9682,"2,750.000000",0.002685,6.026225,0.024551
